In [117]:
import pandas as pd
import numpy as np
import datetime as dt


In [118]:

input_data = pd.read_csv('input.csv', names = ['TimeStamp','Symbol','Quantity','Price'])


In [119]:
## calculating the maximum price of each Ticker, average price, and total volume

### Added a notional column

input_data_edit = input_data
input_data_edit['Notional'] = input_data_edit['Quantity']*input_data_edit['Price']

### calculate the MaxPrice, and quantity

input_data_groups = input_data_edit.groupby('Symbol').agg(Notional = ('Notional', 'sum'), Volume = ('Quantity', 'sum'), MaxPrice = ('Price','max')).reset_index()

### caclulate the WeightedAveragePrice

input_data_groups['WeightedAveragePrice'] = (input_data_groups['Notional']/input_data_groups['Volume']).round(0).astype(int)

#input_data_groups.head()


In [120]:
# calculating the Max Time between ticker trades

## sort values and then calculate time diff

input_data_edit2 = input_data_edit.sort_values(by = ['Symbol','TimeStamp'])

### remove the diff from the last timestamp of a ticker and the first timestamp of the next ticker

input_data_edit2['same_ticker'] = (input_data_edit2['Symbol'] != input_data_edit2['Symbol'].shift()).astype(int)
input_data_edit2['diff'] = input_data_edit2['TimeStamp'].diff()

input_data_edit2 = input_data_edit2[input_data_edit2['same_ticker'] != 1]


## calculate max time diff

max_diff = input_data_edit2.groupby('Symbol').agg(MaxTimeGap = ('diff', 'max')).reset_index()

max_diff['MaxTimeGap'] = max_diff['MaxTimeGap'].astype(int)


In [121]:
## combine the tables and remove unnecessary columns and add 0 to the MaxTimeGap that only had 1 trade

output_data = pd.merge(input_data_groups,max_diff, on = 'Symbol', how = 'outer').drop('Notional',axis=1)
output_data['MaxTimeGap'] = output_data['MaxTimeGap'].replace('',np.nan).fillna(0)

output_data.head()
output_data = output_data[['Symbol','MaxTimeGap','Volume','WeightedAveragePrice','MaxPrice']]

output_data.to_csv('output.csv', index=False,header=False)


